# Reconstructing LandMark Data

We need to use the [LandMark Map data](http://www.landmarkmap.org/map/) for our Land Rights layer on GFW.
These data are private and we can not copy them. We can build the layer based on the way landmark build theirs.

The data are held on an ArcGis MapServer, e.g. [http://gis.wri.org/server/rest/services/LandMark/comm_ind_Documented/MapServer](http://gis.wri.org/server/rest/services/LandMark/comm_ind_Documented/MapServer)

MapServer Query [documentation](https://developers.arcgis.com/rest/services-reference/query-map-service-layer-.htm).

In [31]:
import folium
import json
import requests
from pprint import pprint
from pyproj import Proj, transform

In [10]:
basemap = "Mapbox Bright"

map = folium.Map(location=[28.29, -16.6], zoom_start=3, tiles=basemap)

In [73]:
map.add_tile_layer?

In [74]:

ind_documented = "http://gis.wri.org/server/rest/services/LandMark/comm_ind_Documented/MapServer/tile/{z}/{y}/{x}"
ind_not_documented = "http://gis.wri.org/server/rest/services/LandMark/comm_ind_NotDocumented/MapServer/tile/{z}/{y}/{x}"
ind_formal = "http://gis.wri.org/server/rest/services/LandMark/comm_ind_FormalLandClaim/MapServer/{z}/{y}/{x}"
ind_customary = "http://gis.wri.org/server/rest/services/LandMark/comm_ind_CustomaryTenure/MapServer/tile/{z}/{y}/{x}"


com_land_documented = "http://gis.wri.org/server/rest/services/LandMark/comm_comm_Documented/MapServer/tile/{z}/{y}/{x}"
com_undocumented = "http://gis.wri.org/server/rest/services/LandMark/comm_comm_NotDocumented/MapServer/tile/{z}/{y}/{x}"
com_tenure = "http://gis.wri.org/server/rest/services/LandMark/comm_comm_CustomaryTenure/MapServer/tile/{z}/{y}/{x}"
formal_lc = "http://gis.wri.org/server/rest/services/LandMark/comm_comm_FormalLandClaim/MapServer/tile/{z}/{y}/{x}"


map.add_tile_layer(tiles=ind_documented, name="Indigeouns Land: Documented", attr='LandMark', max_zoom=11, detect_retina=True)
map.add_tile_layer(tiles=ind_not_documented, name="Indigeouns Land: Not Documented", attr='LandMark', max_zoom=11, detect_retina=True)
map.add_tile_layer(tiles=ind_formal, name="Indigenous Land: Formal Tenure", attr='LandMark', max_zoom=11, detect_retina=True)
map.add_tile_layer(tiles=ind_customary, name="Indigenous Land: Customary Tenure", attr='LandMark', max_zoom=11, detect_retina=True)

map.add_tile_layer(tiles=com_land_documented, name="Community Land: Documented", attr='LandMark', max_zoom=11, detect_retina=True)
map.add_tile_layer(tiles=com_undocumented, name="Community Land: Undocumented", attr='LandMark', max_zoom=11, detect_retina=True)
map.add_tile_layer(tiles=com_tenure, name="Community Land: Custom Tenure", attr='LandMark', max_zoom=11, detect_retina=True)
map.add_tile_layer(tiles=formal_lc, name="Community Land: Formal Land Claim", attr='LandMark', max_zoom=11, detect_retina=True)


map.add_child(folium.LatLngPopup())

We also need to return metadata for this layer, and expose it via a pop-up. LandMark appear to query every Arc Layer on click to see if content exists.

An example of an original (unmodified) query from the Landmark site is below.

I have added a lat/long popup to the above to grab coordinates from the map for examples/testing.

In [136]:
example_query = ("http://gis.wri.org/server/rest/services/LandMark/comm_ind_Documented/MapServer/1/query?"
                "f=json&returnGeometry=true&spatialRel=esriSpatialRelIntersects&maxAllowableOffset=2445&"
                "geometry=%7B%22xmin%22%3A-12558297.303392%2C%22ymin%22%3A4411155.852665369%2C%22xmax%22%3A"
                "-12528945.484530501%2C%22ymax%22%3A4440507.671526869%2C%22spatialReference%22%3A%7B%22wkid%"
                "22%3A102100%7D%7D&geometryType=esriGeometryEnvelope&inSR=102100&outFields=OBJECTID%2CCountry"
                "%2CIdentity%2CForm_Rec%2CDoc_Status%2CStat_Date%2CStat_Note%2CISO_Code%2CName%2CCategory%2C"
                "Ethncty_1%2CEthncty_2%2CEthncty_3%2CPopulatn%2CPop_Source%2CPop_Year%2CArea_Ofcl%2CArea_GIS%2C"
                "Scale%2CMethod%2CData_Ctrb%2CData_Src%2CData_Date%2CAdd_Note%2CMore_info%2CUpl_Date%2C"
                "Shape_Length%2CShape_Area&outSR=102100")

r = requests.get(example_query)
print(f"Response code {r.status_code}")
pprint(r.json())

Response code 200
{'displayFieldName': 'Name',
 'features': [{'attributes': {'Add_Note': None,
                              'Area_GIS': 48931.569827286236,
                              'Area_Ofcl': 48932.1255,
                              'Category': 'Indian Reservation',
                              'Country': 'United States of America',
                              'Data_Ctrb': 'Publicly available',
                              'Data_Date': '2015',
                              'Data_Src': 'U.S. Census Bureau',
                              'Doc_Status': 'Documented',
                              'Ethncty_1': None,
                              'Ethncty_2': None,
                              'Ethncty_3': None,
                              'Form_Rec': 'Acknowledged by govt',
                              'ISO_Code': 'USA',
                              'Identity': 'Indigenous',
                              'Method': None,
                              'More_info': 'https://w

Let's break this request down and improve it. Currently it gives back un-necessary info, and works via an envelope geometry. We want to change this to be a simpler intersect against a point geometry in epsg:4326 format, so it's ready to work with our map.

Also, since we dont know in advance which layer our data will be in, we need to query every layer/layer_id combination until we find features. This is the approach that LandMark seems to have taken.

In [137]:
# different_wms = "comm_comm_Documented"
# different_layer = 1

# example_query = (f"http://gis.wri.org/server/rest/services/LandMark/{different_wms}/MapServer/"
#                  f"{different_layer}/query?"
#                   "f=json&returnGeometry=false&spatialRel=esriSpatialRelIntersects&maxAllowableOffset=2445&"
#                   "geometry=%7B%22xmin%22%3A-12558297.303392%2C%22ymin%22%3A4411155.852665369%2C%22xmax%22%3A"
#                   "-12528945.484530501%2C%22ymax%22%3A4440507.671526869%2C%22spatialReference%22%3A%7B%22wkid%"
#                   "22%3A102100%7D%7D&geometryType=esriGeometryEnvelope&inSR=102100&outFields=OBJECTID%2CCountry"
#                   "%2CIdentity%2CForm_Rec%2CDoc_Status%2CStat_Date%2CStat_Note%2CISO_Code%2CName%2CCategory%2C"
#                   "Ethncty_1%2CEthncty_2%2CEthncty_3%2CPopulatn%2CPop_Source%2CPop_Year%2CArea_Ofcl%2CArea_GIS%2C"
#                   "Scale%2CMethod%2CData_Ctrb%2CData_Src%2CData_Date%2CAdd_Note%2CMore_info%2CUpl_Date%2C"
#                   "Shape_Length%2CShape_Area&outSR=102100")

# r = requests.get(example_query)
# r.status_code
# print(f"{len(r.json().get('features'))} features not returned from querying the {different_wms}/{different_layer} layer")

In [112]:
# different_wms = "comm_ind_Documented"
# different_layer = 1

# example_query = (f"http://gis.wri.org/server/rest/services/LandMark/{different_wms}/MapServer/"
#                  f"{different_layer}/query?"
#                   "f=json&returnGeometry=false&spatialRel=esriSpatialRelIntersects&maxAllowableOffset=2445&"
#                   "geometry=%7B%22xmin%22%3A-12558297.303392%2C%22ymin%22%3A4411155.852665369%2C%22xmax%22%3A"
#                   "-12528945.484530501%2C%22ymax%22%3A4440507.671526869%2C%22spatialReference%22%3A%7B%22wkid%"
#                   "22%3A102100%7D%7D&geometryType=esriGeometryEnvelope&inSR=102100&outFields=OBJECTID%2CCountry"
#                   "%2CIdentity%2CForm_Rec%2CDoc_Status%2CStat_Date%2CStat_Note%2CISO_Code%2CName%2CCategory%2C"
#                   "Ethncty_1%2CEthncty_2%2CEthncty_3%2CPopulatn%2CPop_Source%2CPop_Year%2CArea_Ofcl%2CArea_GIS%2C"
#                   "Scale%2CMethod%2CData_Ctrb%2CData_Src%2CData_Date%2CAdd_Note%2CMore_info%2CUpl_Date%2C"
#                   "Shape_Length%2CShape_Area&outSR=102100")

# r = requests.get(example_query)
# r.status_code
# pprint(r.json().get('features'))

In [138]:
# layer = "comm_ind_Documented"
# layer_id = 1
# location = -111.6870, 31.9708  # example location

# eg_query = (f"http://gis.wri.org/server/rest/services/LandMark/{layer}/MapServer/{layer_id}/query?"
#              "f=json&returnGeometry=false&"
#              "spatialRel=esriSpatialRelIntersects&"
#               f"geometry= {location}&"
#               "geometryType=esriGeometryPoint&"
#               "inSR={'wkid': 4326}&"
#               "outFields=OBJECTID,Country,Identity,Form_Rec,Doc_Status,Stat_Date,Stat_Note,ISO_Code,"
#               "Name,Category,Ethncty_1,Ethncty_2,Ethncty_3,Populatn,Pop_Source,Pop_Year,Area_Ofcl,"
#               "Area_GIS,Scale,Method,Data_Ctrb,Data_Src,Data_Date,Add_Note,More_info,Upl_Date,Shape_Length,"
#               "Shape_Area")

# r = requests.get(eg_query)
# r.status_code
# print(r.url)
# pprint(r.json().get('features'))

https://developers.arcgis.com/rest/services-reference/query-map-service-layer-.htm

In [134]:
layers = [
    "comm_ind_Documented",
    "comm_ind_NotDocumented",
    "comm_ind_FormalLandClaim",
    "comm_ind_CustomaryTenure",
    "comm_comm_Documented",
    "comm_comm_NotDocumented",
    "comm_comm_CustomaryTenure",
    "comm_comm_FormalLandClaim"
         ]

layer_ids = [0, 1]

def find_popup_data(lon, lat):
    """
    Try all possible layer/layer_id combinations looking too see if any of the intersects
    contain data in the response. (This is currently what LandMark seems to do on their map site.)
    Return the dictionary object if it exists, ready to put into a pop-up!
    """
    location = lon, lat
    n = 1
    for layer in layers:
        for layer_id in layer_ids:
            
            print(f"try {n} of {len(layer_ids) * len(layers)}")
            n += 1
            test_query = (
                f"http://gis.wri.org/server/rest/services/LandMark/{layer}/MapServer/{layer_id}/query?"
                 "f=json&returnGeometry=false&"
                 "spatialRel=esriSpatialRelIntersects&"
                f"geometry= {location}&"
                 "geometryType=esriGeometryPoint&"
                 "inSR={'wkid': 4326}&"
                 "outFields=OBJECTID,Country,Identity,Form_Rec,Doc_Status,Stat_Date,Stat_Note,ISO_Code,"
                 "Name,Category,Ethncty_1,Ethncty_2,Ethncty_3,Populatn,Pop_Source,Pop_Year,Area_Ofcl,"
                 "Area_GIS,Scale,Method,Data_Ctrb,Data_Src,Data_Date,Add_Note,More_info,Upl_Date,Shape_Length,"
                 "Shape_Area"
            )
            try:
                r = None
                r = requests.get(test_query)
                if len(r.json().get('features')) > 0:
                    popup_json = r.json().get('features')[0].get('attributes')
                    print(f'Found data in {layer}/{layer_id}:')
                    return popup_json
            except:
                pass


In [135]:
find_popup_data(-111.6870, 31.9708)  # example location

try 1 of 16
try 2 of 16
Found data in comm_ind_Documented/1:


{'OBJECTID': 33784,
 'Country': 'United States of America',
 'Identity': 'Indigenous',
 'Form_Rec': 'Acknowledged by govt',
 'Doc_Status': 'Documented',
 'Stat_Date': None,
 'Stat_Note': None,
 'ISO_Code': 'USA',
 'Name': "Tohono O'odham Nation Reservation",
 'Category': 'Reservation',
 'Ethncty_1': None,
 'Ethncty_2': None,
 'Ethncty_3': None,
 'Populatn': None,
 'Pop_Source': None,
 'Pop_Year': None,
 'Area_Ofcl': 1153488.9236,
 'Area_GIS': 1153484.205257292,
 'Scale': 'Unknown',
 'Method': None,
 'Data_Ctrb': 'Publicly available',
 'Data_Src': 'U.S. Census Bureau',
 'Data_Date': '2015',
 'Add_Note': None,
 'More_info': 'https://www.census.gov/geo/maps-data/data/tiger-line.html',
 'Upl_Date': '2015/08/26',
 'Shape_Length': 831532.6191235977,
 'Shape_Area': 16139317739.31186}

Above shows the final working query, within a loop, searching a given lon, lat point for intersecting data. This is ready to put into a pop-up window.